In [5]:
import pathlib
import torch


models = [
    file for file in pathlib.Path("../../runs/bpic2012/20%").rglob("*.best_val_acc.pth")
]
models

[PosixPath('../../runs/bpic2012/20%/20250301.1304.no_constraint/model.epoch_65.best_val_acc.pth'),
 PosixPath('../../runs/bpic2012/20%/20250301.1318.no_constraint/model.epoch_50.best_val_acc.pth'),
 PosixPath('../../runs/bpic2012/20%/20250301.1331.no_constraint/model.epoch_58.best_val_acc.pth'),
 PosixPath('../../runs/bpic2012/20%/20250301.1344.no_constraint/model.epoch_96.best_val_acc.pth')]

In [57]:
from pylon_experiments.model.training.constraints.constraint import (
    constraint_from_string,
)

import numpy as np
import pandas as pd

for model_path in models:
    dataset = model_path.parents[2].name
    percentage = model_path.parents[1].name

    output_path = model_path.parent / model_path.stem

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, weights_only=False).to(device)

    constraint_file = (
        pathlib.Path("../../data/") / dataset / "extracted" / "constraints.txt"
    )

    constraints: dict[str, tuple[any, float]] = {}
    with open(constraint_file) as f:
        lines = f.readlines()
        for line in lines:
            constraint_str, constraint_support = line.split(" - Support: ")

            constraint = constraint_from_string(constraint_str, model.vocab)
            constraint_type_str, remaining = constraint_str.split("[")
            constraint_activities = remaining.removesuffix("]").split(",")
            constraint_activities = [
                constraint_activity.strip()
                + "("
                + str(model.vocab.to_idx(constraint_activity.strip()))
                + ")"
                for constraint_activity in constraint_activities
            ]
            constraint_str = (
                constraint_type_str + "[" + ", ".join(constraint_activities) + "]"
            )

            constraint_support = constraint_support.strip().removesuffix("%")
            constraint_support = float(constraint_support)

            constraints[constraint_str] = (
                constraint,
                constraint_support,
            )

    with open(output_path / "predicted_traces.txt") as f:
        keys = [
            f"{constraint_str} {support}%"
            for constraint_str, (constraint, support) in constraints.items()
        ]

        results = pd.DataFrame(columns=["trace"] + keys)

        lines = f.readlines()
        for line in lines:
            line = line.removeprefix("[").removesuffix("]\n")
            np_trace = np.fromstring(line, dtype=int, sep=" ")
            trace = torch.tensor(np_trace).unsqueeze(0).to(device)

            row = [f"{np_trace}"]

            for constraint_str, (constraint, support) in constraints.items():
                sat = constraint._condition(trace, **constraint._settings.dict()).item()

                row.append(sat)

            results.loc[len(results)] = row

    # results.to_csv(output_path / "constraints_satisfied.csv", index=False)
results

,trace,Absence2[A_PREACCEPTED(6)] 100.0%,Init[A_SUBMITTED(4)] 100.0%,Exactly1[A_PARTLYSUBMITTED(5)] 100.0%,Exactly1[A_SUBMITTED(4)] 100.0%,"Chain Succession[A_SUBMITTED(4), A_PARTLYSUBMITTED(5)] 100.0%",Absence2[A_FINALIZED(10)] 100.0%,Absence2[A_ACCEPTED(8)] 100.0%,Absence2[A_DECLINED(22)] 100.0%,Absence2[O_CREATED(11)] 89.01%,...,Absence2[O_SENT(12)] 89.01%,Absence[W_Nabellen incomplete dossiers(26)] 87.41%,Absence[O_ACCEPTED(18)] 82.86%,Absence[A_REGISTERED(16)] 82.84%,Absence[A_ACTIVATED(19)] 82.84%,Absence[A_APPROVED(17)] 82.84%,Absence[O_CANCELLED(20)] 79.67%,Absence[A_CANCELLED(23)] 78.55%,Absence[O_SENT_BACK(14)] 75.14%,Absence[W_Valideren aanvraag(15)] 75.14%
0,[ 4 5 6 7 7 7 7 7 7 7 7 7 7 7 9 ...,True,True,True,True,True,True,True,True,False,...,False,False,False,True,False,True,False,True,True,False
1,[ 4 5 6 24 6 24 6 7 24 7 7 7 7 7 7 ...,False,True,True,True,True,True,True,True,False,...,False,True,True,True,True,True,True,True,True,False
2,[ 4 5 6 7 7 7 9 10 11 12 13 7 13 13 20 ...,True,True,True,True,True,True,True,True,False,...,False,False,True,False,True,False,False,True,True,False
3,[ 4 5 6 7 7 7 7 7 9 10 11 12 13 7 13 ...,True,True,True,True,True,True,True,True,True,...,True,False,True,True,True,True,True,True,True,False
4,[ 4 5 6 7 7 7 7 7 7 7 7 7 7 7 7 ...,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,[4 5 6 3],True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
232,[4 5 6 3],True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
233,[4 5 6 3],True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
234,[4 5 6 3],True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


Absence2[A_PREACCEPTED(6)] 100.0%                                0.711864
Init[A_SUBMITTED(4)] 100.0%                                      1.000000
Exactly1[A_PARTLYSUBMITTED(5)] 100.0%                            1.000000
Exactly1[A_SUBMITTED(4)] 100.0%                                  1.000000
Chain Succession[A_SUBMITTED(4), A_PARTLYSUBMITTED(5)] 100.0%    1.000000
Absence2[A_FINALIZED(10)] 100.0%                                 0.991525
Absence2[A_ACCEPTED(8)] 100.0%                                   1.000000
Absence2[A_DECLINED(22)] 100.0%                                  0.995763
Absence2[O_CREATED(11)] 89.01%                                   0.868644
Absence2[O_SELECTED(9)] 89.01%                                   0.809322
Absence2[O_SENT(12)] 89.01%                                      0.868644
Absence[W_Nabellen incomplete dossiers(26)] 87.41%               0.775424
Absence[O_ACCEPTED(18)] 82.86%                                   0.885593
Absence[A_REGISTERED(16)] 82.84%      